Install required depencencies

In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install transformers datasets

In [ ]:
!pip3 install --pre torch torchvision torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121

In [ ]:
! pip install accelerate -U

Based onn robert-base model

In [58]:
from transformers import BartTokenizer, BartForSequenceClassification
from transformers import Trainer, TrainingArguments
import torch
tokenizer = BartTokenizer.from_pretrained('roberta-base')
model = BartForSequenceClassification.from_pretrained('roberta-base', num_labels=3)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RobertaTokenizer'. 
The class this function is called from is 'BartTokenizer'.
You are using a model of type roberta to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.
Some weights of BartForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['decoder.layers.5.self_attn.q_proj.bias', 'encoder.layers.8.self_attn.v_proj.weight', 'decoder.layers.10.encoder_attn.out_proj.weight', 'encoder.layers.0.self_attn.k_proj.bias', 'decoder.layers.9.self_attn_layer_norm.weight', 'decoder.layers.11.encoder_attn.q_proj.bias', 'decoder.layers.1.self_attn.q_proj.bias', 'decoder.layers.4.encoder_attn.q_proj.weight', 'decoder.layers.8.self_attn.out_proj.weight', 'encoder.layer

donwload dataset.

In [59]:
from datasets import load_dataset
dataset = load_dataset("Jasontth/climate_fever_plus")
train_db = dataset["train"]
val_db = dataset["valid"]

Encode dataset

In [67]:
max_length = 512

def transform_labels(examples):
    tokenized_inputs = tokenizer(examples['claim'], examples['evidence'], truncation=True, padding='max_length', max_length=max_length)

    labels = [1 if label == 'SUPPORTS' else 0 if label == "REFUTES" else 2 for label in examples['evidence_label']]

    tokenized_inputs['labels'] = labels
    return tokenized_inputs


encoded_train_dataset = train_db.map(transform_labels, batched=True)
encoded_val_dataset = val_db.map(transform_labels, batched=True)


Map: 100%|██████████| 768/768 [00:00<00:00, 3764.02 examples/s]


Check the data structure

In [68]:
encoded_train_dataset[0]

{'claim': 'A recent study led by Lawrence Livermore National Laboratory climate scientist Ben Santer found that while the models ran hot, the ‘overestimation’ was ‘partly due to systematic deficiencies in some of the post-2000 external forcings used in the model simulations.’',
 'evidence': 'While it is believed that the modeled climate-cooling-effects from the mass of soot injected into the stratosphere by 100 firestorms (one to five teragrams) would have been detectable with technical instruments in WWII, five percent of that would not have been possible to observe at that time.',
 'evidence_label': 'NOT_ENOUGH_INFO',
 'input_ids': [0,
  250,
  485,
  892,
  669,
  30,
  6226,
  29711,
  4321,
  496,
  17815,
  2147,
  9744,
  1664,
  764,
  1334,
  303,
  14,
  150,
  5,
  3092,
  2075,
  2131,
  6,
  5,
  44,
  711,
  7067,
  7110,
  39431,
  17,
  27,
  21,
  44,
  711,
  7755,
  352,
  528,
  7,
  20552,
  28262,
  11,
  103,
  9,
  5,
  618,
  12,
  17472,
  6731,
  33147,
  103

In [89]:
from transformers import DataCollatorForLanguageModeling
# data_collactor = DataCollatorForLanguageModeling(
#     tokenizer=tokenizer
# )
training_args = TrainingArguments(
    output_dir = "/content/our-model",
    learning_rate=2e-5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 10,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    push_to_hub = False
)

Clear GPU memory

In [90]:
import torch
torch.cuda.empty_cache()

Train the model. Then save it to claim_evidence_fever
The result: 
{'train_runtime': 68292.5642, 'train_samples_per_second': 0.964, 'train_steps_per_second': 0.06, 'train_loss': 0.6050088688007836, 'epoch': 10.0}


In [91]:
trainer = Trainer(
    model=model,
    args=training_args,
    # data_collator=data_collactor,
    train_dataset=encoded_train_dataset,
    eval_dataset=encoded_val_dataset
)

trainer.train()
trainer.save_model("./model/cliam_evidence_fever")
model.save_pretrained("./model/cliam_evidence_fever")

 10%|█         | 412/4120 [1:56:28<13:17:29, 12.90s/it]

{'eval_loss': 0.8905285000801086, 'eval_runtime': 414.2259, 'eval_samples_per_second': 1.854, 'eval_steps_per_second': 0.116, 'epoch': 1.0}


 12%|█▏        | 500/4120 [2:18:33<15:50:26, 15.75s/it]  

{'loss': 0.9389, 'learning_rate': 1.7572815533980585e-05, 'epoch': 1.21}


 20%|██        | 824/4120 [3:55:08<13:48:19, 15.08s/it]

...

{'loss': 0.3603, 'learning_rate': 5.825242718446603e-07, 'epoch': 9.71}


100%|██████████| 4120/4120 [18:58:00<00:00, 14.95s/it]

{'eval_loss': 1.2284237146377563, 'eval_runtime': 503.8808, 'eval_samples_per_second': 1.524, 'eval_steps_per_second': 0.095, 'epoch': 10.0}


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight'].
100%|██████████| 4120/4120 [18:58:12<00:00, 16.58s/it]


{'train_runtime': 68292.5642, 'train_samples_per_second': 0.964, 'train_steps_per_second': 0.06, 'train_loss': 0.6050088688007836, 'epoch': 10.0}


Evaluate the model

In [16]:
! pip install -U scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/60.4 kB ? eta -:--:--
     --------------------------------- ------ 51.2/60.4 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 60.4/60.4 kB 1.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   - -------------------------------------- 0.3/9.1 MB 6.5 MB/s eta 0:00:02
   -- ------------------------------------- 0.5/9.1 MB 7.4 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/9.1 MB 4.7 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.1 MB 6.1 MB/s eta 0:00:02
   ------- -------------------------------- 1.7/9.1 MB 7.1 MB/s eta 0:00:02
   --------- ------------------------------ 2.2/9.1 MB 7.8 MB/s eta 0:00:01
   ----------- ---------------------------- 2.7/9.1 MB 8.2 MB/s eta 0:00:01
   ------------- -------------------------- 3.2/9.1 MB 8.5 MB/s eta 0:00:01
   ---------------- ----------------------- 3.7/9.1 MB 8.7 MB/s eta 0:00:01
   ------------

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("./model/cliam_evidence_fever")
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# Load dataset
test_dataset = load_dataset("Jasontth/climate_fever_plus", split='valid')

# encode data
def encode(examples):
    return tokenizer(examples['claim'], examples['evidence'], truncation=True, padding='max_length', max_length=512)

encoded_test_dataset = test_dataset.map(encode, batched=True)
encoded_test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'evidence_label'])

# create DataLoader
test_dataloader = DataLoader(encoded_test_dataset, batch_size=16)

# Model prediction
model.eval()
predictions = []

for batch in test_dataloader:
    with torch.no_grad():
        inputs = {k: v.to(model.device) for k, v in batch.items() if k in tokenizer.model_input_names}
        outputs = model(**inputs)
        batch_predictions = outputs.logits.argmax(dim=-1)
        predictions.extend(batch_predictions.cpu().numpy())

# Transfer label to numbers
label_mapping = {'SUPPORTS': 0, 'REFUTES': 1, 'NOT_ENOUGH_INFO': 2}
true_labels = [label_mapping[label] for label in test_dataset["evidence_label"]]

# evaluate
accuracy = accuracy_score(true_labels, predictions)
precision = precision_score(true_labels, predictions, average='macro')
recall = recall_score(true_labels, predictions, average='macro')
f1 = f1_score(true_labels, predictions, average='macro')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


c:\Users\Jalon\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jalon\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Jalon\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Jalon\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is 

Accuracy: 0.5559895833333334
Precision: 0.3828540305010893
Recall: 0.38870712593581885
F1 Score: 0.3228682425695881
